# Chapter 4 Advanced SQL for SQLite

<div id="toc"></div>

## 4.1 Modifying Data

### Inserting Records

In [ ]:
insert into table (column_list) values (value_list);

* Inserting One Row

In [ ]:
insert into foods (name, type_id) values ('Cinnamon Bobka', 1);

In [ ]:
select * from foods where name='Cinnamon Bobka';

In [ ]:
select max(id) from foods;

In [ ]:
select last_insert_rowid();

In [ ]:
insert into foods values(NULL, 1, 'Blueberry Bobka');
select * from foods where name like '%Bobka';

* Inserting a Set of Rows

In [ ]:
insert into foods
values (null,
       (select id from food_types where name='Bakery'),
       'Blackberry Bobka');
select * from foods where name like '%Bobka';

In [ ]:
insert into foods
select last_insert_rowid()+1, type_id, name from foods
where name='Chocolate Bobka';
select * from foods where name like '%Bobka';

* Inserting Multiple Rows

In [ ]:
create table foods2 (id int, type_id int, name text);
insert into foods2 select * from foods;
select count(*) from foods2;

In [ ]:
create table foods2 as select * from foods;
select count(*) from list;

In [ ]:
sqlite> select max(id) from foods; max(id)

In [ ]:
sqlitex insert into foods values (416, 1, 'Chocolate Bobka'); 
SOL error: PRIMARY KEY must be unique

In [ ]:
create temp table list as
select f.name food, t.name name,
       (select count(episode_id)
        from foods_episodes where food_id=f.id) episodes
from foods f, food_types t
where f.type_id=t.id;
select * from list;

### Updating Records

In [ ]:
update foods set name='CHOCOLATE BOBKA'
where name='Chocolate Bobka';
select * from foods where name like 'CHOCOLATE%';

### Deleting Records

In [ ]:
delete from foods where name='CHOCOLATE BOBKA';

## 4.2 Data Integrity

In [ ]:
create table contacts (
id integer primary key,
name text not null collate nocase,
phone text not null default 'UNKNOWN',
unique (name,phone) );

### Entity Integrity

* Unique Constraints

In [ ]:
insert into contacts (name,phone) values ('Jerry','UNKNOWN');
insert into contacts (name) values ('Jerry');
insert into contacts (name,phone) values ('Jerry', '555-1212');

* Primary Key Constraints

In [ ]:
select rowid, oid,_rowid_,id, name, phone from contacts;

In [ ]:
create table maxed_out(id integer primary key autoincrement, x text);
insert into maxed_out values (9223372036854775807, 'last one');
select * from sqlite_sequence;

In [ ]:
drop table maxed_out;
create table maxed_out(id integer primary key autoincrement, x text);
insert into maxed_out values(10, 'works');
select * from sqlite_sequence;

In [ ]:
insert into maxed_out values(9, 'works');
select * from sqlite_sequence;

In [ ]:
insert into maxed_out values (9, 'fails');

In [ ]:
insert into maxed_out values (null, 'should be 11');
select * from maxed_out;

In [ ]:
select * from sqlite_sequence;

In [ ]:
create table pkey(x text, y text, primary key(x,y));
insert into pkey values ('x','y');
insert into pkey values ('x','x');
select rowid, x, y from pkey;

### Domain Integrity

* Default Values

In [ ]:
insert into contacts (name) values ('Jerry');
select * from contacts;

In [ ]:
create table times ( id int,
  date not null default current_date,
  time not null default current_time,
  timestamp not null default current_timestamp );
insert into times (id) values (1);
insert into times (id) values (2);
select * from times;

* NOT NULL Constraints

In [ ]:
insert into contacts (phone) values ('555-1212');

* Check Constraints

In [ ]:
create table contacts
( id integer primary key,
name text not null collate nocase,
phone text not null default 'UNKNOWN',
unique (name,phone),
check (length(phone)>=7) );

* Foreign Key Constraints

In [ ]:
create table foo
( x integer,
y integer check (y>x),
z integer check (z>abs(y)) );

In [ ]:
insert into foo values (-2, -1, 2);
insert into foo values (-2, -1, 1);

In [ ]:
update foo set y=-3 where x=-3;

In [ ]:
create table foods(
  id integer primary key,
  type_id integer references food_types(id)
  on delete restrict
  deferrable initially deferred,
  name text );

* Collations

In [ ]:
insert into contacts (name,phone) values ('JERRY','555-1212');

### Storage Classes

In [ ]:
drop table domain;
create table domain(x);
insert into domain values (3.142);
insert into domain values ('3.142');
insert into domain values (3142);
insert into domain values (x'3142');
insert into domain values (null);
select rowid, x, typeof(x) from domain;

In [ ]:
select typeof(3.14), typeof('3.14'),
       typeof(314), typeof(x'3142'), typeof(NULL);

### Views

In [ ]:
create view name as select-stmt;

In [ ]:
select f.name, ft.name, e.name
from foods f
inner join food_types ft on f.type_id=ft.id
inner join foods_episodes fe on f.id=fe.food_id
inner join episodes e on fe.episode_id=e.id;

In [ ]:
create view details as
select f.name as fd, ft.name as tp, e.name as ep, e.season as ssn
from foods f
inner join food_types ft on f.type_id=ft.id
inner join foods_episodes fe on f.id=fe.food_id
inner join episodes e on fe.episode_id=e.id;

In [ ]:
select fd as Food, ep as Episode
        from details where ssn=7 and tp like 'Drinks';

In [ ]:
drop view name;

### Indexes

In [ ]:
SELECT * FROM foods WHERE name='JujyFruit';

In [ ]:
create index [unique] index_name on table_name (columns)

In [ ]:
create table foo(a text, b text);
create unique index foo_idx on foo(a,b);
insert into foo values ('unique', 'value');
insert into foo values ('unique', 'value2');
insert into foo values ('unique', 'value');

* Collations

In [ ]:
create index foods_name_idx on foods (name collate nocase);

In [ ]:
sqlite> .indices foods foods_name_idx
For more information, you can use the .schema shell command as well: 
sqlite> .schema foods 
CREATE TABLE foods( 
    id integer primary key, 
    type_id integer, name text );
CREATE INDEX foods_name_idx on foods (name COLLATE NOCASE);

* Index Utilization

In [ ]:
column {=|>|>=|<=|<} 
expression expression 
{=|>|>=|<=|<} column 
column IN (expression-list) 
column IN (subquery)

In [ ]:
create table foo (a,b,c,d);

create index foo_idx on foo (a,b,c,d);

select * from foo where a=1 and b=2 and d=3;

select * from foo where a>1 and b=2 and c=3 and d=4;

select * from foo where a=1 and b>2 and c=3 and d=4;

### Triggers

In [ ]:
create [temp|temporary] trigger name
[beforejafter] [insert|delete|update|update of columns] on table action

* Update Triggers

In [ ]:
create temp table log(x);

create temp trigger foods_update_log update of name on foods
begin
  insert into log values('updated foods: new name=' || new.name);
end;

begin;
update foods set name='JUJYFRUIT' where name='JujyFruit';
select * from log;
rollback;

In [ ]:
create temp table log(x);

create temp trigger foods_update_log after update of name on foods
begin
  insert into log values('updated foods: new name=' || new.name);
end;

begin;
update foods set name='JUJYFRUIT' where name='JujyFruit';
rollback;

* Error Handling

In [ ]:
raise(resolution, error_message);

* Updatable Views

In [ ]:
create view foods_view as
  select f.id fid, f.name fname, t.id tid, t.name tname
  from foods f, food_types t;

In [ ]:
create trigger on_update_foods_view
instead of update on foods_view
for each row
begin
   update foods set name=new.fname where id=new.fid;
   update food_types set name=new.tname where id=new.tid;
end;

In [ ]:
.echo on
-- Update the view within a transaction
begin;
update foods_view set fname='Whataburger', tname='Fast Food' where fid=413;
-- Now view the underlying rows in the base tables:
select * from foods f, food_types t where f.type_id=t.id and f.id=413;
-- Roll it back
rollback;
-- Now look at the original record:
select * from foods f, food_types t where f.type_id=t.id and f.id=413;
begin;
update foods_view set fname='Whataburger', tname='Fast Food' where fid=413;
select * from foods f, food_types t where f.type_id=t.id and f.id=413;
rollback;
select * from foods f, food_types t where f.type_id=t.id and f.id=413;

## 4.3 Transactions

## Transaction Scopes

In [ ]:
begin;
delete from foods;
rollback;
select count(*) from foods;

### Conflict Resolution

In [ ]:
sqlite> update foods set id=800-id;
SOL error: PRIMARY KEY must be unique

### Database Locks

In [ ]:
create table test as select * from foods;
create unique index test_idx on test(id);
alter table test add column modified text not null default 'no';
select count(*) from test where modified='no';

update or fail test set id=800-id, modified='yes';

select count(*) from test where modified='yes';

drop table test;

create temp table cast(name text unique on conflict rollback);
insert into cast values ('Jerry');
insert into cast values ('Elaine');
insert into cast values ('Kramer');

begin;
insert into cast values('Jerry');
commit;

### Deadlocks 

In [ ]:
create trigger foods_update_trg
before update of type_id on foods
begin
  select case
     when (select id from food_types where id=new.type_id) is null
     then raise( abort,
                 'Foreign Key Violation: foods.type_id is not in food_types.id')
  end;
end

explain query plan select * from foods where id = 145;

### Transaction Types 

In [ ]:
begin [ deferred | immediate | exclusive ] transaction;

## 4.4 Database Administration

### Attaching Databases

In [ ]:
attach [databasej/ilenarne as database_name;

In [ ]:
sqlite> attach database '/tmp/db' as db2; sqlite> select * from db2.foo;
x bar

In [ ]:
sqlite> select * from main.foods limit 2; 

In [ ]:
sqlite> create temp table foo as select * from food_types limit 3; 
sqlite> select * from temp.foo;

### Cleaning Databases

In [ ]:
reindex collation_name; 
reindex table_name|index_name;

### Database Configuration

* The Connection Cache Size

In [ ]:
sqlite> pragma cache_size;
cache size 2000
sqlite> pragma cache_size=l0000; sqlite> pragma cache_size;
cache size 10000

* Getting Database Information

In [ ]:
sqlite> pragma database_list;

In [ ]:
sqlite> create index foods_name_type_idx on foods(name,type_id); 

In [ ]:
sqlite> pragma index_info(foods_name_type_idx);

In [ ]:
sqlite> pragma index_list(foods); 
seq

In [ ]:
sqlite> pragma table_info(foods);

* Synchronous Writes

* Temporary Storage

* Page Size, Encoding, and Autovacuum

* Debugging

### The System Catalog

In [ ]:
sqlite> select type, name, rootpage from sqlite_master;

In [ ]:
sqlite> select sql from sqlite_master where name='foods_update_trg';

In [ ]:
sqlite> explain query plan select * from foods where id = 145; order

### Viewing Query Plans

## 4.5 Summary

SQL may be a simple language to use, but there is quite a bit of it, and it’s taken us two chapters just to introduce the major concepts for SQLite’s implementation of SQL.  
But that shouldn’t be too surprising, because it is the sole interface through which to interact with a relational database.  
Whether you are a casual user, system administrator, or developer, you have to know SQL if you are going to work with a relational database.  
If you are programming with SQLite, then you should be off to a good start on the SQL side of things.  
Now you need to know a little about how SQLite goes about executing all of these commands.  
This is where Chapter 5 should prove useful.  
It will introduce you to the API and show you how it works in relation to the way SQLite functions internally.   
